We start by initializing our LLM. 

In [2]:
from langchain_ollama import OllamaLLM

# For normal, accurate responses (low temperature)
llm = OllamaLLM(model="llama3:instruct", temperature=0.0)

# For unique, creative responses (higher temperature)
creative_llm = OllamaLLM(model="llama3:instruct", temperature=0.9)


We will be taking an `article` _draft_ and using LangChain to generate various useful items around this article. We'll be creating:

1. An article title
2. An article description
3. Editor advice where we will insert an additional paragraph in the article
4. A thumbnail / hero image for our article.

In [12]:
article = """

Spider-Man swings into AGE OF REVELATION in this week's Marvel Comics!

Unfriendly and unstable! X years later, the X-Virus decimated New York—but Spider-Man won't stop fighting in RADIOACTIVE SPIDER-MAN (2025) #1 by Joe Kelly and Kev Walker. In a desperate gamble, Peter Parker doses himself with lethal radiation to hold the infection at bay. Survival comes at a cost. While this may not kill Peter Parker, it sure messes him up. This is the most dangerous, mutated Spider-Man ever—and he's not alone.

Behold Peter Parker's next evolution in RADIOACTIVE SPIDER-MAN (2025) #1, on sale this Wednesday, October 22!

RADIOACTIVE SPIDER-MAN (2025) #1 cover by Giuseppe Camuncoli
RADIOACTIVE SPIDER-MAN (2025) #1 cover by Giuseppe Camuncoli
AGE OF REVELATION rages on in Jed MacKay and Netho Diaz's X-MEN: BOOK OF REVELATION (2025) #1, where the Game of Power commences! X years later, after the assassination of one of his prized Choristers, Revelation welcomes a new mutant to his capital city of Philadelphia. But the capital of the Revelation Territories hides many dangers, not the least of which are her rival Choristers—and Fabian Cortez will allow no one to challenge him for Revelation's favor. But this new mutant has a secret ally—the Ghost of Philadelphia!

What happened to Wolverine? X years later, in THE LAST WOLVERINE (2025) #1 by Saladin Ahmed and Edgar Salazar, the people of Vancouver cheer a new hero: the Wonderful Wolverine, AKA the Wendigo, Logan's last student. But what happened to Logan? A secret from Wolverine's past will set the Last Wolverine on a mission to uphold his mentor's legacy… unless a dire threat burns it all down first!

Elsewhere, Kid Omega's all grown up for Tony Fleecs and Andres Genolet's OMEGA KIDS (2025) #1! X years later, Quentin Quire protects the dream of mutant supremacy and independence as head of the Revelation Territories' spy network. But when a conspiracy threatens the mutant utopia, Quire and his psychic students will have to distinguish friend from foe. Has Quentin got what it takes to carry the dream forward, or will the next generation replace the former revolutionary?

Over in the new Ultimate Universe, Miles Morales faces a showdown with the Maker's Council for ULTIMATE SPIDER-MAN: INCURSION (2025) #5 by Deniz Camp, Cody Ziglar, and Jonas Scharf! The clock is ticking for Miles to escape the Ultimate Universe with his baby sister! How will Miles leave a mark on the Ultimate Universe? And how will the Ultimate Universe leave a mark on Miles?! Don't miss the exciting conclusion…and what it bodes for the future of both universes!

Meanwhile, now that the Ultimates are using the Immortus Engine to its full capacity, Doom's dream of the Ultimate Fantastic Four may finally be realized in Deniz Camp and Phil Noto's ULTIMATES (2024) #17. 

Then, as the Ultimate Universe hurtles towards its conclusion, behold the last will and testament of [REDACTED] in ULTIMATE SPIDER-MAN (2024) #22 by Jonathan Hickman and Marco Checchetto!

See Spider-Man's radioactive evolution, meet the Last Wolverine, escape the Ultimate Universe with Miles Morales, and more by picking up the latest titles this week! From fresh comics to collections and even Marvel Unlimited, find the full list of new releases here!
"""


## Preparing our Prompts

Prompts for chat agents are at a minimum broken up into three components, those are:

* System prompt: this provides the instructions to our LLM on how it must behave, what it's objective is, etc.

* User prompt: this is a user written input.

* AI prompt: this is the AI generated output. When representing a conversation, previous generations will be inserted back into the next prompt and become part of the broader _chat history_.

```
You are a helpful AI assistant, you will do XYZ.    | SYSTEM PROMPT

User: Hi, what is the capital of Australia?         | USER PROMPT
AI: It is Canberra                                  | AI PROMPT
User: When is the best time to visit?               | USER PROMPT
```

LangChain provides us with _templates_ for each of these prompt types. By using templates we can insert different inputs to the template, modifying the prompt based on the provided inputs.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(template = "You are an AI assistant called {name} that helps generate article titles.")

# The user prompt, parameterized over "article"
user_prompt = HumanMessagePromptTemplate.from_template(template= """You are tasked with creating a name for an article.
The article is here for you to examine: {article}

The name should be based on the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided."""
)

# Combine prompts using ChatPromptTemplate
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])


In [14]:
print(first_prompt.input_variables)

['article', 'name']


In [15]:
print(user_prompt.format(article = "Test Article"))

content='You are tasked with creating a name for an article.\nThe article is here for you to examine: Test Article\n\nThe name should be based on the context of the article.\nBe creative, but make sure the names are clear, catchy,\nand relevant to the theme of the article.\n\nOnly output the article name, no other explanation or\ntext can be provided.' additional_kwargs={} response_metadata={}


In [16]:
formatted = first_prompt.format_messages(article=" Test Article", name = "Jarvis")

for msg in formatted:
    print(f"{msg.type.capitalize()}: {msg.content}")

System: You are an AI assistant called Jarvis that helps generate article titles.
Human: You are tasked with creating a name for an article.
The article is here for you to examine:  Test Article

The name should be based on the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.


`ChatPromptTemplate` also prefixes each individual message with it's role, ie `System:`, `Human:`, or `AI:`.

We can chain together our `first_prompt` template and the `llm` object we defined earlier to create a simple LLM chain. This chain will perform the steps **prompt formatting > llm generation > get output**.

We'll be using **L**ang**C**hain **E**xpression **L**anguage (LCEL) to construct our chain. We define our inputs with the first dictionary segment (ie `{"article": lambda x: x["article"]}`) and then we use the pipe operator (`|`) to say that the output from the left of the pipe will be fed into the input to the right of the pipe.

In [17]:
chain_one = (
    {"article": lambda x: x["article"], "name": lambda x: x["name"]} | first_prompt | creative_llm | {"article_title": lambda x: x}
)

In [18]:
result = chain_one.invoke({"article": article, "name": "Jarvis"})
print(result)


{'article_title': '"Marvel\'s Mutated Mayhem: Evolution Unleashed!"'}


In [19]:
chain_two = (
    {"article": lambda x: x["article"], "name": lambda x: x["name"]} | first_prompt | llm | {"article_title": lambda x: x}
)

In [20]:
result = chain_two.invoke({"article": article, "name": "Jarvis"})
print(result)

{'article_title': '"Marvel\'s Mutant Mayhem: A New Era Unleashed"'}
